In [2]:
import xml.etree.ElementTree as ET
import pandas as pd

def get_items(response):
    # 응답 데이터를 텍스트로 변환
    xml_string = response.text  # 또는 response.content.decode('utf-8')
    
    root = ET.fromstring(xml_string)
    item_list = []
    
    # 모든 row 엘리먼트 찾기
    rows = root.findall('row')
    
    # 각 row 엘리먼트에서 데이터 추출
    for row in rows:
        elements = row.findall('*')
        data = {}
        for element in elements:
            tag = element.tag.strip()
            text = element.text.strip() if element.text else None
            data[tag] = text
        item_list.append(data)
    
    return item_list

# items_list = get_items(res)
# items = pd.DataFrame(items_list)
# items.head()

In [3]:
import requests
import datetime
import certifi
import ssl

def get_data(statbl_id, datacycle_cd, cls_id):
    url = "https://www.reb.or.kr/r-one/openapi/SttsApiTblData.do?"
    service_key = "37f7a534514349d48f2cf77dc2691da3"
    

    call_code = "Key=" + service_key + "&" +\
        "pIndex=1&pSize=1000&" +\
        "STATBL_ID=" + statbl_id + "&" +\
        "DTACYCLE_CD=" + datacycle_cd +\
        "&START_WRTTIME=2015&END_WRTTIME=2025" +"&" +\
        "CLS_ID=" + cls_id
    
    print(url+call_code)
    res = requests.get(url + call_code, verify=certifi.where())
    return res

In [4]:
STATBL_ID = "A_2024_00060"
# cls_id_list = ["530088", "530089", "530090"]
cls_id_list = ["520045"]
#cls_id_list = ["500009"]
tx_list = []
for cls_id in cls_id_list:
    res = get_data(STATBL_ID, "MM", cls_id)
    tx_list += get_items(res)
    
len(tx_list)
apt_tx = pd.DataFrame(tx_list)
apt_tx

https://www.reb.or.kr/r-one/openapi/SttsApiTblData.do?Key=37f7a534514349d48f2cf77dc2691da3&pIndex=1&pSize=1000&STATBL_ID=A_2024_00060&DTACYCLE_CD=MM&START_WRTTIME=2015&END_WRTTIME=2025&CLS_ID=520045


,STATBL_ID,DTACYCLE_CD,WRTTIME_IDTFR_ID,GRP_ID,GRP_NM,CLS_ID,CLS_NM,ITM_ID,ITM_NM,DTA_VAL,UI_NM,GRP_FULLNM,CLS_FULLNM,ITM_FULLNM,WRTTIME_DESC
0,A_2024_00060,MM,201501,None,None,520045,고양시,100001,가격,272972,천원,None,경기>경의권>고양시,가격,2015년 1월
1,A_2024_00060,MM,201502,None,None,520045,고양시,100001,가격,274864,천원,None,경기>경의권>고양시,가격,2015년 2월
2,A_2024_00060,MM,201503,None,None,520045,고양시,100001,가격,276831,천원,None,경기>경의권>고양시,가격,2015년 3월
3,A_2024_00060,MM,201504,None,None,520045,고양시,100001,가격,278868,천원,None,경기>경의권>고양시,가격,2015년 4월
4,A_2024_00060,MM,201505,None,None,520045,고양시,100001,가격,280840,천원,None,경기>경의권>고양시,가격,2015년 5월
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,A_2024_00060,MM,202404,None,None,520045,고양시,100001,가격,521156,천원,None,경기>경의권>고양시,가격,2024년 4월
112,A_2024_00060,MM,202405,None,None,520045,고양시,100001,가격,520891,천원,None,경기>경의권>고양시,가격,2024년 5월
113,A_2024_00060,MM,202406,None,None,520045,고양시,100001,가격,520808,천원,None,경기>경의권>고양시,가격,2024년 6월
114,A_2024_00060,MM,202407,None,None,520045,고양시,100001,가격,520723.16,천원,None,경기>경의권>고양시,가격,2024년 7월


In [5]:
goyang_apt_tx = apt_tx.drop(['DTACYCLE_CD', 'STATBL_ID','GRP_ID','GRP_NM','CLS_ID','ITM_ID','GRP_FULLNM', 'CLS_FULLNM','ITM_FULLNM'], axis=1)
goyang_apt_tx.isnull().sum()

WRTTIME_IDTFR_ID    0
CLS_NM              0
ITM_NM              0
DTA_VAL             0
UI_NM               0
WRTTIME_DESC        0
dtype: int64

In [6]:
# 수치형식 변환
goyang_apt_tx['WRTTIME_IDTFR_ID'] = goyang_apt_tx['WRTTIME_IDTFR_ID'].astype(int)
goyang_apt_tx['DTA_VAL'] = goyang_apt_tx['DTA_VAL'].astype(float)
goyang_apt_tx['DTA_VAL'] = goyang_apt_tx['DTA_VAL']/10

In [7]:
# 항목별 범주형 데이터 확인
# goyang_apt_tx['WRTTIME_IDTFR_ID'].value_counts()
# goyang_apt_tx['CLS_NM'].value_counts()
# goyang_apt_tx['ITM_NM'].value_counts()
goyang_apt_tx['UI_NM'].value_counts()

UI_NM
천원    116
Name: count, dtype: int64

In [8]:
# 수치형 데이터 기본 EDA
goyang_apt_tx['DTA_VAL'].describe()

count      116.000000
mean     40654.615571
std      11614.179047
min      27297.200000
25%      30551.225000
50%      34217.100000
75%      52074.437000
max      61405.700000
Name: DTA_VAL, dtype: float64

In [9]:
goyang_apt_tx

,WRTTIME_IDTFR_ID,CLS_NM,ITM_NM,DTA_VAL,UI_NM,WRTTIME_DESC
0,201501,고양시,가격,27297.200000,천원,2015년 1월
1,201502,고양시,가격,27486.400000,천원,2015년 2월
2,201503,고양시,가격,27683.100000,천원,2015년 3월
3,201504,고양시,가격,27886.800000,천원,2015년 4월
4,201505,고양시,가격,28084.000000,천원,2015년 5월
...,...,...,...,...,...,...
111,202404,고양시,가격,52115.600000,천원,2024년 4월
112,202405,고양시,가격,52089.100000,천원,2024년 5월
113,202406,고양시,가격,52080.800000,천원,2024년 6월
114,202407,고양시,가격,52072.316000,천원,2024년 7월


In [51]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd

# 데이터 전처리
goyang_monthly_tx = goyang_apt_tx.copy()
goyang_monthly_tx['WRTTIME_IDTFR_ID'] = pd.to_datetime(goyang_monthly_tx['WRTTIME_IDTFR_ID'], format='%Y%m')
monthly_avg_tx = goyang_monthly_tx.groupby('WRTTIME_IDTFR_ID')['DTA_VAL'].mean().reset_index()

# 초기 데이터 필터링
filtered_data = monthly_avg_tx

# y축 최솟값과 최댓값 계산
y_min = filtered_data['DTA_VAL'].min()
y_min_with_margin = y_min - (y_min * 0.03)  # 3% 마진 추가
y_max = filtered_data['DTA_VAL'].max()
y_max_with_margin = y_max + (y_max * 0.03)  # 3% 마진 추가

# Plotly 그래프 생성
fig = px.line(filtered_data, 
              x='WRTTIME_IDTFR_ID', 
              y='DTA_VAL', 
              markers=True, 
              labels={'WRTTIME_IDTFR_ID': '거래월', 'DTA_VAL': '평균매매가격(만원)'})

# 선 그래프 색상 및 굵기 설정
fig.update_traces(line=dict(color='#007BFF', width=7),  # 밝고 비비드한 파란색
                  marker=dict(size=11, color='#007BFF', symbol='circle'))  # 큰 빨간색 마커 추가

# y축 라벨 형식 수정
tickvals = list(range(int(y_min_with_margin // 1000 * 1000), int(y_max_with_margin // 1000 * 1000) + 1000, 1000))  # 리스트로 변환
ticktext = [f"{int(value // 10000)}억{value % 10000:04}만원" for value in tickvals]  # 형식 수정

# 그래프 레이아웃 업데이트 (크기, 제목, 축 스타일, 배경색 등)
fig.update_layout(
    title={
        'text': '고양시 아파트 월별 평균매매가격',
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 28, 'family': 'Montserrat', 'color': '#2B3E50'}
    },
    #template="plotly_white",  # 화이트 테마 적용
    xaxis=dict(
        title='거래월 (2022~2024년)',
        tickfont=dict(size=16, family='Roboto', color='#3E3E3E'),  # 큰 글꼴 적용
        titlefont=dict(size=20, family='Roboto', color='#2B3E3E'),
        tickmode='linear',
        dtick='M1',  # 매월 눈금 표시
        #showgrid=True,  # 세로선 표시
        #gridcolor='rgba(211, 211, 211, 0.5)',  # 연한 회색 점선
        gridwidth=1
    ),
    yaxis=dict(
        title='평균매매가격(만원)',
        tickfont=dict(size=16, family='Roboto', color='#3E3E3E'),
        titlefont=dict(size=20, family='Roboto', color='#2B3E50'),
        tickmode='array',  # 수동으로 tick 설정
        tickvals=tickvals,  # y축 눈금 위치
        ticktext=ticktext,  # y축 눈금 라벨
        # showgrid=True,  # 가로선 표시
        # gridcolor='rgba(211, 211, 211, 0.5)',  # 연한 회색 점선
        # gridwidth=1,
        range=[y_min_with_margin, y_max_with_margin]  # y축의 최솟값과 최댓값에 마진 추가
    ),
    width=1200,  # 그래프 너비
    height=700,  # 그래프 높이
    plot_bgcolor='white',  # 그래프 배경색을 깔끔한 흰색으로
    paper_bgcolor='#F8F8FF',   # 전체 배경색을 가벼운 회색
    xaxis_tickformat='%Y년 %m월',
    xaxis_tickangle=45,  # x축 눈금 기울기
    margin=dict(l=40, r=40, t=80, b=60),  # 여백 조정
)

# y축 점선 추가
for y_value in range(int(y_min_with_margin//1000*1000), int(y_max_with_margin//1000*1000), 1000):
    fig.add_shape(type='line',
                  xref='paper', x0=0, x1=1,
                  yref='y', y0=y_value, y1=y_value,
                  line=dict(color="rgba(180,180,180,0.7)", width=1, dash="dot"))  # 점선과 굵기 설정

# x축 점선 추가
for x_value in filtered_data['WRTTIME_IDTFR_ID']:
    fig.add_shape(type='line',
                  yref='paper', y0=0, y1=1,
                  xref='x', x0=x_value, x1=x_value,
                  line=dict(color="rgba(180,180,180,0.7)", width=1, dash="dot"))  # 점선과 굵기 설정

# 포인트 위에 텍스트 추가
for i in range(len(filtered_data)):
    deal_amount = filtered_data['DTA_VAL'].iloc[i]
    billions = int(deal_amount // 10000)
    ten_thousands = int(deal_amount % 10000)

    # 형식 수정: 5억2000만원
    if billions > 0:
        label = f'{billions}억{ten_thousands:04}만원'  # 4자리 수로 포맷
    else:
        label = f'{ten_thousands}만원'
    
    fig.add_annotation(
        x=filtered_data['WRTTIME_IDTFR_ID'].iloc[i],
        y=deal_amount,
        text=label,
        showarrow=False,
        textangle=-25,  # 오른쪽으로 기울이기
        xanchor='left',  # 텍스트의 시작점을 왼쪽으로 설정
        yanchor='bottom',  # 텍스트의 시작점을 아래쪽으로 설정
        font=dict(size=12, color="black"),  # 폰트 크기와 색상 설정
        xshift=10  # 오른쪽으로 이동
    )

# Dash 애플리케이션 초기화
app = dash.Dash(__name__)

# 레이아웃 정의
app.layout = html.Div([
    dcc.Graph(id='line-chart', figure=fig)
])

# 애플리케이션 실행
if __name__ == '__main__':
    app.run_server(debug=True)


OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.

In [ ]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd

# 데이터 전처리
goyang_monthly_tx = goyang_apt_tx.copy()
goyang_monthly_tx['WRTTIME_IDTFR_ID'] = pd.to_datetime(goyang_monthly_tx['WRTTIME_IDTFR_ID'], format='%Y%m')
monthly_avg_tx = goyang_monthly_tx.groupby('WRTTIME_IDTFR_ID')['DTA_VAL'].mean().reset_index()

# 초기 데이터 필터링
filtered_data = monthly_avg_tx

# y축 최솟값과 최댓값 계산
y_min = filtered_data['DTA_VAL'].min()
y_min_with_margin = y_min - (y_min * 0.03)  # 3% 마진 추가
y_max = filtered_data['DTA_VAL'].max()
y_max_with_margin = y_max + (y_max * 0.03)  # 3% 마진 추가

# Plotly 그래프 생성
fig = px.line(filtered_data, 
              x='WRTTIME_IDTFR_ID', 
              y='DTA_VAL', 
              markers=False, 
              labels={'WRTTIME_IDTFR_ID': '거래월', 'DTA_VAL': '평균매매가격(만원)'})

# 선 그래프 색상 및 굵기 설정
fig.update_traces(line=dict(color='#007BFF', width=4))

# y축 라벨 형식 수정
tickvals = list(range(int(y_min_with_margin // 1000 * 1000), int(y_max_with_margin // 1000 * 1000) + 1000, 1000))  # 리스트로 변환
ticktext = [f"{int(value // 10000)}억{value % 10000:04}만원" for value in tickvals]  # 형식 수정

# 그래프 레이아웃 업데이트 (크기, 제목, 축 스타일, 배경색 등)
fig.update_layout(
    title={
        'text': '고양시 아파트 월별 평균매매가격',
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 28, 'family': 'Montserrat', 'color': '#2B3E50'}
    },
    #template="plotly_white",  # 화이트 테마 적용
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        title='거래월 (2022~2024년)',
        tickfont=dict(size=12, family='Arial', color='rgb(82, 82, 82)'),  # 큰 글꼴 적용
        titlefont=dict(size=16, family='Arial', color='#2B3E3E'),
        tickmode='linear',
        dtick='M1',  # 매월 눈금 표시
        #showgrid=True,  # 세로선 표시
        #gridcolor='rgba(211, 211, 211, 0.5)',  # 연한 회색 점선
        gridwidth=1
    ),
    yaxis=dict(
        zeroline=False,
        showline=True,
        showgrid=False,
        linecolor='rgb(204, 204, 204)',
        title='평균매매가격(만원)',
        tickfont=dict(size=12, family='Arial', color='#3E3E3E'),
        titlefont=dict(size=16, family='Arial', color='#2B3E50'),
        tickmode='array',  # 수동으로 tick 설정
        tickvals=tickvals,  # y축 눈금 위치
        ticktext=ticktext,  # y축 눈금 라벨
        # showgrid=True,  # 가로선 표시
        # gridcolor='rgba(211, 211, 211, 0.5)',  # 연한 회색 점선
        # gridwidth=1,
        range=[y_min_with_margin, y_max_with_margin]  # y축의 최솟값과 최댓값에 마진 추가
    ),
    width=1200,  # 그래프 너비
    height=700,  # 그래프 높이
    plot_bgcolor='white',  # 그래프 배경색을 깔끔한 흰색으로
    #paper_bgcolor='#F8F8FF',   # 전체 배경색을 가벼운 회색
    paper_bgcolor='white',   # 전체 배경색을 가벼운 회색
    xaxis_tickformat='%Y년 %m월',
    xaxis_tickangle=45,  # x축 눈금 기울기
    margin=dict(l=40, r=40, t=80, b=60),  # 여백 조정
)

# # y축 점선 추가
# for y_value in range(int(y_min_with_margin//1000*1000), int(y_max_with_margin//1000*1000), 1000):
#     fig.add_shape(type='line',
#                   xref='paper', x0=0, x1=1,
#                   yref='y', y0=y_value, y1=y_value,
#                   line=dict(color="rgba(180,180,180,0.7)", width=1, dash="dot"))  # 점선과 굵기 설정
# 
# # x축 점선 추가
# for x_value in filtered_data['WRTTIME_IDTFR_ID']:
#     fig.add_shape(type='line',
#                   yref='paper', y0=0, y1=1,
#                   xref='x', x0=x_value, x1=x_value,
#                   line=dict(color="rgba(180,180,180,0.7)", width=1, dash="dot"))  # 점선과 굵기 설정

# 포인트 위에 텍스트 추가
# for i in range(len(filtered_data)):
#     deal_amount = filtered_data['DTA_VAL'].iloc[i]
#     billions = int(deal_amount // 10000)
#     ten_thousands = int(deal_amount % 10000)
# 
#     # 형식 수정: 5억2000만원
#     if billions > 0:
#         label = f'{billions}억{ten_thousands:04}만원'  # 4자리 수로 포맷
#     else:
#         label = f'{ten_thousands}만원'
#     
#     fig.add_annotation(
#         x=filtered_data['WRTTIME_IDTFR_ID'].iloc[i],
#         y=deal_amount,
#         text=label,
#         showarrow=False,
#         textangle=-25,  # 오른쪽으로 기울이기
#         xanchor='left',  # 텍스트의 시작점을 왼쪽으로 설정
#         yanchor='bottom',  # 텍스트의 시작점을 아래쪽으로 설정
#         font=dict(size=12, color="black"),  # 폰트 크기와 색상 설정
#         xshift=10  # 오른쪽으로 이동
#     )

# Dash 애플리케이션 초기화
app = dash.Dash(__name__)

# 레이아웃 정의
app.layout = html.Div([
    dcc.Graph(id='line-chart', figure=fig)
])

# 애플리케이션 실행
if __name__ == '__main__':
    app.run_server(debug=True)


In [ ]:
from bokeh.io import curdoc
from bokeh.models import ColumnDataSource, LabelSet, NumeralTickFormatter, Range1d, DatetimeTickFormatter
from bokeh.plotting import figure
from bokeh.layouts import column
import pandas as pd
import numpy as np

# 데이터 전처리
goyang_monthly_tx = goyang_apt_tx.copy()
goyang_monthly_tx['WRTTIME_IDTFR_ID'] = pd.to_datetime(goyang_monthly_tx['WRTTIME_IDTFR_ID'], format='%Y%m')
monthly_avg_tx = goyang_monthly_tx.groupby('WRTTIME_IDTFR_ID')['DTA_VAL'].mean().reset_index()

# Bokeh 데이터 소스 설정
source = ColumnDataSource(data=dict(
    date=monthly_avg_tx['WRTTIME_IDTFR_ID'],
    avg_price=monthly_avg_tx['DTA_VAL'],
))

# 그래프 기본 설정
p = figure(title='고양시 아파트 월별 평균매매가격',
           x_axis_label='거래월 (2022~2024년)',
           y_axis_label='평균매매가격(만원)',
           x_axis_type='datetime', 
           width=1200, height=700)

# y축 포맷 조정
p.yaxis.formatter = NumeralTickFormatter(format="0,0")

# y축과 x축에 마진 추가
y_min = monthly_avg_tx['DTA_VAL'].min() - monthly_avg_tx['DTA_VAL'].min() * 0.03
y_max = monthly_avg_tx['DTA_VAL'].max() + monthly_avg_tx['DTA_VAL'].max() * 0.03
p.y_range = Range1d(y_min, y_max)

# y축 포맷팅
ticks = list(range(int(monthly_avg_tx['DTA_VAL'].min()//1000*1000), int(monthly_avg_tx['DTA_VAL'].max()//1000*1000) + 1000, 1000))
tick_labels = [f"{int(tick // 10000)}억{tick % 10000:04}만원" for tick in ticks]
p.yaxis.ticker = ticks
p.yaxis.major_label_overrides = {tick: label for tick, label in zip(ticks, tick_labels)}

# 라인 그래프 추가
p.line('date', 'avg_price', source=source, line_width=8, color='blue', legend_label='평균매매가격', line_dash='solid')

# 포인트 위에 텍스트 어노테이션 추가
labels = []
for i, row in monthly_avg_tx.iterrows():
    deal_amount = row['DTA_VAL']
    billions = int(deal_amount // 10000)
    ten_thousands = int(deal_amount % 10000)

    # 5억2000만원 형식으로 변환
    if billions > 0:
        label = f'{billions}억{ten_thousands:04}만원'
    else:
        label = f'{ten_thousands}만원'

    labels.append(label)

# 어노테이션 추가
label_source = ColumnDataSource(data=dict(
    x=monthly_avg_tx['WRTTIME_IDTFR_ID'],
    y=monthly_avg_tx['DTA_VAL'],
    labels=labels,
))

labels_set = LabelSet(x='x', y='y', text='labels', level='glyph', x_offset=10, y_offset=10, 
                      source=label_source, text_font_size="10pt", text_color="black",
                      angle=-24.7)  # 오른쪽으로 기울이기

p.add_layout(labels_set)

# X축의 월별 표시 설정
p.xaxis.ticker = np.arange(monthly_avg_tx['WRTTIME_IDTFR_ID'].min().timestamp()*1000,
                            monthly_avg_tx['WRTTIME_IDTFR_ID'].max().timestamp()*1000,
                            30*24*60*60*1000)  # 1개월 단위로 설정

# X축 레이블 포맷 설정
p.xaxis.formatter = DatetimeTickFormatter(
    months=["%Y년 %m월"],
)

# 레이블 기울기 조정
p.xaxis.major_label_orientation = 45  # 45도 기울이기

# 레이아웃 구성
layout = column(p)

# Bokeh 서버 실행
curdoc().add_root(layout)


In [ ]:
show(layout)

In [ ]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd

# 데이터 전처리
goyang_monthly_tx = goyang_apt_tx.copy()
goyang_monthly_tx['WRTTIME_IDTFR_ID'] = pd.to_datetime(goyang_monthly_tx['WRTTIME_IDTFR_ID'], format='%Y%m')

# Plotly 그래프 생성 (지역구별 라인 그래프)
fig = px.line(goyang_monthly_tx, 
              x='WRTTIME_IDTFR_ID', 
              y='DTA_VAL', 
              color='CLS_NM',  # 지역구별로 색상 구분
              markers=True, 
              labels={'WRTTIME_IDTFR_ID': '거래월', 'DTA_VAL': '평균매매가격(만원)', 'CLS_NM': '지역구'})

# 선 그래프 색상 및 굵기 설정
fig.update_traces(line=dict(width=7),  # 선 굵기 통일
                  marker=dict(size=11))  # 마커 크기 통일

# y축 라벨 형식 수정
y_min = goyang_monthly_tx['DTA_VAL'].min()
y_min_with_margin = y_min - (y_min * 0.03)  # 3% 마진 추가
y_max = goyang_monthly_tx['DTA_VAL'].max()
y_max_with_margin = y_max + (y_max * 0.03)  # 3% 마진 추가

tickvals = list(range(int(y_min_with_margin // 1000 * 1000), int(y_max_with_margin // 1000 * 1000) + 1000, 1000))  # 리스트로 변환
ticktext = [f"{int(value // 10000)}억{value % 10000:04}만원" for value in tickvals]  # 형식 수정

# 그래프 레이아웃 업데이트 (크기, 제목, 축 스타일, 배경색 등)
fig.update_layout(
    title={
        'text': '고양시 아파트 지역구별 월별 평균매매가격',
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 28, 'family': 'Montserrat', 'color': '#2B3E50'}
    },
    xaxis=dict(
        title='거래월 (2022~2024년)',
        tickfont=dict(size=16, family='Roboto', color='#3E3E3E'),
        titlefont=dict(size=20, family='Roboto', color='#2B3E3E'),
        tickmode='linear',
        dtick='M1',
        gridwidth=1
    ),
    yaxis=dict(
        title='평균매매가격(만원)',
        tickfont=dict(size=16, family='Roboto', color='#3E3E3E'),
        titlefont=dict(size=20, family='Roboto', color='#2B3E50'),
        tickmode='array',
        tickvals=tickvals,
        ticktext=ticktext,
        range=[y_min_with_margin, y_max_with_margin]
    ),
    width=1200,
    height=700,
    plot_bgcolor='white',
    paper_bgcolor='#F8F8FF',
    xaxis_tickformat='%Y년 %m월',
    xaxis_tickangle=45,
    margin=dict(l=40, r=40, t=80, b=60),
    legend=dict(
        title='지역구',  # 범례 제목
        font=dict(size=25),
        orientation='v',
        yanchor="top",
        y=0.95,  # 그래프 상단에 배치
        xanchor="right",
        x=0.95,  # 그래프 오른쪽에 배치
        bgcolor='rgba(255, 255, 255, 0.5)'  # 반투명한 배경 적용
    )
)

# y축 점선 추가
for y_value in range(int(y_min_with_margin//1000*1000), int(y_max_with_margin//1000*1000), 1000):
    fig.add_shape(type='line',
                  xref='paper', x0=0, x1=1,
                  yref='y', y0=y_value, y1=y_value,
                  line=dict(color="rgba(180,180,180,0.7)", width=1, dash="dot"))  # 점선과 굵기 설정

# x축 점선 추가
for x_value in goyang_monthly_tx['WRTTIME_IDTFR_ID'].unique():
    fig.add_shape(type='line',
                  yref='paper', y0=0, y1=1,
                  xref='x', x0=x_value, x1=x_value,
                  line=dict(color="rgba(180,180,180,0.7)", width=1, dash="dot"))  # 점선과 굵기 설정

# Dash 애플리케이션 초기화
app = dash.Dash(__name__)

# 레이아웃 정의
app.layout = html.Div([
    dcc.Graph(id='line-chart', figure=fig)
])

# 애플리케이션 실행
if __name__ == '__main__':
    app.run_server(debug=True)


In [10]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd

# 데이터 전처리
goyang_monthly_tx = goyang_apt_tx.copy()
goyang_monthly_tx['WRTTIME_IDTFR_ID'] = pd.to_datetime(goyang_monthly_tx['WRTTIME_IDTFR_ID'], format='%Y%m')
monthly_avg_tx = goyang_monthly_tx.groupby('WRTTIME_IDTFR_ID')['DTA_VAL'].mean().reset_index()

# 초기 데이터 필터링
filtered_data = monthly_avg_tx

# y축 최솟값과 최댓값 계산
y_min = filtered_data['DTA_VAL'].min()
y_min_with_margin = y_min - (y_min * 0.03)  # 3% 마진 추가
y_max = filtered_data['DTA_VAL'].max()
y_max_with_margin = y_max + (y_max * 0.03)  # 3% 마진 추가

# Plotly 그래프 생성
fig = px.line(filtered_data, 
              x='WRTTIME_IDTFR_ID', 
              y='DTA_VAL', 
              markers=False, 
              line_shape='spline',
              #labels={'WRTTIME_IDTFR_ID': '거래월', 'DTA_VAL': '평균매매가격(만원)'}
              )

# 선 그래프 색상 및 굵기 설정
fig.update_traces(line=dict(color='#007BFF', width=5))  # 밝고 비비드한 파란색

# y축 라벨 형식 수정
tickvals = list(range(int(y_min_with_margin // 10000 * 10000), int(y_max_with_margin // 10000 * 10000) + 1000, 10000))  # 리스트로 변환
ticktext = [f"{int(value // 10000)}억" for value in tickvals]  # 형식 수정

# 그래프 레이아웃 업데이트 (크기, 제목, 축 스타일, 배경색 등)
fig.update_layout(
    # title={
    #     'text': '고양시 아파트 월별 평균매매가격',
    #     'y': 0.95,
    #     'x': 0.5,
    #     'xanchor': 'center',
    #     'yanchor': 'top',
    #     'font': {'size': 28, 'family': 'Montserrat', 'color': '#2B3E50'}
    # },
    #template="plotly_white",  # 화이트 테마 적용
    xaxis=dict(
        showline=True,
        showticklabels=False,
        title=None,
        # title='거래월 (2022~2024년)',
        # tickfont=dict(size=16, family='Roboto', color='#3E3E3E'),  # 큰 글꼴 적용
        # titlefont=dict(size=20, family='Roboto', color='#2B3E3E'),
        # tickmode='linear',
        # dtick='M1',  # 매월 눈금 표시
        showgrid=False,  # 세로선
        # #gridcolor='rgba(211, 211, 211, 0.5)',  # 연한 회색 점선
        # gridwidth=1
    ),
    yaxis=dict(
        showline=True,
        showticklabels=True,
        title=None,
        tickfont=dict(size=25, family='Verdana', color='gray'),
        titlefont=dict(size=20, family='Roboto', color='#2B3E50'),
        tickmode='array',  # 수동으로 tick 설정
        tickvals=tickvals,  # y축 눈금 위치
        ticktext=ticktext,  # y축 눈금 라벨
        # showgrid=True,  # 가로선 표시
        # gridcolor='rgba(211, 211, 211, 0.5)',  # 연한 회색 점선
        # gridwidth=1,
        range=[y_min_with_margin, y_max_with_margin]  # y축의 최솟값과 최댓값에 마진 추가
    ),
    width=750,  # 그래프 너비
    height=500,  # 그래프 높이
    plot_bgcolor='white',  # 그래프 배경색을 깔끔한 흰색으로
    paper_bgcolor='white',   # 전체 배경색을 가벼운 회색
    xaxis_tickformat='%Y년',
    #xaxis_tickangle=45,  # x축 눈금 기울기
    margin=dict(l=40, r=40, t=80, b=60),  # 여백 조정
)

# y축 점선 추가
for y_value in range(int(y_min_with_margin//10000*10000), int(y_max_with_margin//10000*10000)+10000, 10000):
    fig.add_shape(type='line',
                  xref='paper', x0=0, x1=1,
                  yref='y', y0=y_value, y1=y_value,
                  line=dict(color="rgba(180,180,180,0.7)", width=1, dash="solid"))  # 점선과 굵기 설정

# x축 점선 추가
# for x_value in filtered_data['WRTTIME_IDTFR_ID']:
#     fig.add_shape(type='line',
#                   yref='paper', y0=0, y1=1,
#                   xref='x', x0=x_value, x1=x_value,
#                   line=dict(color="rgba(180,180,180,0.7)", width=1, dash="dot"))  # 점선과 굵기 설정

# 포인트 위에 텍스트 추가
# for i in range(len(filtered_data)):
#     deal_amount = filtered_data['DTA_VAL'].iloc[i]
#     billions = int(deal_amount // 10000)
#     ten_thousands = int(deal_amount % 10000)
# 
#     # 형식 수정: 5억2000만원
#     if billions > 0:
#         label = f'{billions}억{ten_thousands:04}만원'  # 4자리 수로 포맷
#     else:
#         label = f'{ten_thousands}만원'
#     
#     fig.add_annotation(
#         x=filtered_data['WRTTIME_IDTFR_ID'].iloc[i],
#         y=deal_amount,
#         text=label,
#         showarrow=False,
#         textangle=-25,  # 오른쪽으로 기울이기
#         xanchor='left',  # 텍스트의 시작점을 왼쪽으로 설정
#         yanchor='bottom',  # 텍스트의 시작점을 아래쪽으로 설정
#         font=dict(size=12, color="black"),  # 폰트 크기와 색상 설정
#         xshift=10  # 오른쪽으로 이동
#     )

# Dash 애플리케이션 초기화
app = dash.Dash(__name__)

# 레이아웃 정의
app.layout = html.Div([
    dcc.Graph(id='line-chart', figure=fig)
])

# 애플리케이션 실행
if __name__ == '__main__':
    app.run_server(debug=True)


In [43]:
help(dash.html.Font)

Help on class Font in module dash.html.Font:

class Font(dash.development.base_component.Component)
 |  Font(children=None, id=undefined, n_clicks=undefined, n_clicks_timestamp=undefined, disable_n_clicks=undefined, key=undefined, accessKey=undefined, className=undefined, contentEditable=undefined, dir=undefined, draggable=undefined, hidden=undefined, lang=undefined, role=undefined, spellCheck=undefined, style=undefined, tabIndex=undefined, title=undefined, loading_state=undefined, **kwargs)
 |
 |  A Font component.
 |  Font is a wrapper for the <font> HTML5 element.
 |  For detailed attribute info see:
 |  https://developer.mozilla.org/en-US/docs/Web/HTML/Element/font
 |
 |  Keyword arguments:
 |
 |  - children (a list of or a singular dash component, string or number; optional):
 |      The children of this component.
 |
 |  - id (string; optional):
 |      The ID of this component, used to identify dash components in
 |      callbacks. The ID needs to be unique across all of the com